## code for testing functionality of the NN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import Layers, loaders

In [3]:
# validation

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Layers.BCVNN(image_channels=3, filter_dimension=3).to(device)

# Dummy RGB input: batch of 8 images, 3×256×256
dummy = torch.randn(8, 3, 256, 256).to(device)
out = model(dummy)

print("Output shape:", out.shape)
# Expected: [8, 101]

Output shape: torch.Size([8, 101])


## validation with synthetic, random dataset

In [7]:
# validation with synthetic dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import Layers, loaders
from tqdm import tqdm

In [8]:
# make synthetic dataset
num_samples = 1000
num_classes = 101
image_size = 256
batch_size = 64

# Fake RGB images (float32 in [0, 1])
X = torch.randn(num_samples, 3, image_size, image_size)
# Random integer labels between 0 and num_classes-1
y = torch.randint(0, num_classes, (num_samples,))

# Wrap as TensorDataset for easy DataLoader batching
dataset = TensorDataset(X, y)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [9]:
# Initialize model, loss, optimizer 
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Layers.BCVNN(image_channels=3, filter_dimension=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
# Training loop
model.train()
for epoch in range(2):  # a couple of epochs for sanity check
    total_loss = 0.0
    for batch_idx, (images, labels) in tqdm(enumerate(loader), desc="training batches"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)

        # Verify output shape
        assert outputs.shape == (images.size(0), num_classes), \
            f"Unexpected output shape {outputs.shape}"

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if (batch_idx + 1) % 5 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1} complete. Avg Loss: {avg_loss:.4f}")

training batches: 5it [01:08, 13.24s/it]

Epoch 1, Batch 5, Loss: 4.6220


training batches: 10it [02:59, 20.77s/it]

Epoch 1, Batch 10, Loss: 4.6110


training batches: 15it [04:26, 18.29s/it]

...

Epoch 2, Batch 10, Loss: 4.6139


training batches: 15it [04:38, 20.40s/it]

Epoch 2, Batch 15, Loss: 4.6190


training batches: 16it [04:49, 18.12s/it]

Epoch 2 complete. Avg Loss: 4.6136


In [12]:
# Evaluation sanity check
model.eval()
with torch.no_grad():
    images, labels = next(iter(loader))
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    preds = outputs.argmax(dim=1)

    print("\nValidation batch:")
    print("Output shape:", outputs.shape)
    print("Predictions:", preds[:10])
    print("True labels:", labels[:10])


Validation batch:
Output shape: torch.Size([64, 101])
Predictions: tensor([15, 15, 15, 15, 15, 15, 15, 15, 15, 15])
True labels: tensor([ 87,  45,  42, 100,  84,   6,  42,  25,  31,  44])
